In [9]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [14]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Title

In [15]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 10, # max 10
        'include_log' : True, # alphaS
        'include_log2' : True
    }, 
    'fit_type' : 'ma', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFK', #'FKfpi', 'FpiFpi', 'FKFK'
    'bias_correct' : True,
    
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
}

In [16]:
for F2 in ['FKFpi', 'FpiFpi', 'FKFK']:
    for vol in [0, 10]:
        for include_alphaS in [False, True]:
            for include_logSq in [False, True]:

                t0 = time.time()

                # Loop through combinations
                p_dict['order']['vol'] = vol
                p_dict['order']['include_log'] = include_alphaS
                p_dict['order']['include_log2'] = include_logSq
                p_dict['F2'] = F2

                if vol > 0:
                    include_FV = True
                else:
                    include_FV = False

                name = F2
                if include_FV:
                    name = name + '_FV'
                if include_alphaS:
                    name = name + '_alphaS'
                if include_logSq:
                    name = name + '_logSq'

                print name
                print "(model, F2, Vol, alphaS, logSq):", (p_dict['fit_type'], p_dict['F2'], include_FV, include_alphaS, include_logSq)

                # Load data
                data_loader = dl.data_loader()
                fit_data = data_loader.get_fit_data()


                bootstrapper = bs.bootstrapper(fit_data, prior=None, order=p_dict['order'], F2=p_dict['F2'], 
                                               fit_type=p_dict['fit_type'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct'])

                new_prior = bootstrapper.create_prior_from_fit()
                data_loader.save_prior(new_prior, fit_type=p_dict['fit_type'], F2=p_dict['F2'], 
                                       include_FV=include_FV, include_alphaS=include_alphaS, include_logSq=include_logSq)

                t1 = time.time()


                print "\nTotal time (s): ", t1 - t0, "\n"

FKFpi
(model, F2, Vol, alphaS, logSq): ('ma', 'FKFpi', False, False, False)
Using default prior.
{'chiral': 1.0,'spacing': 1.0}
0.0(5.0) 0.0(5.0)
{'chiral': 1.05,'spacing': 1.0}
0.0(5.2) 0.0(5.0)
{'chiral': 1.0,'spacing': 1.05}
0.0(5.0) 0.0(5.2)
{'chiral': 0.95,'spacing': 1.0499999999999998}
0.0(4.8) 0.0(5.2)
{'chiral': 0.8999999999999999,'spacing': 1.0749999999999997}
0.0(4.5) 0.0(5.4)
{'chiral': 0.8999999999999999,'spacing': 1.0249999999999997}
0.0(4.5) 0.0(5.1)
{'chiral': 0.8499999999999996,'spacing': 1.0124999999999997}
0.0(4.2) 0.0(5.1)
{'chiral': 0.7499999999999996,'spacing': 1.0874999999999995}
0.0(3.7) 0.0(5.4)
{'chiral': 0.6249999999999991,'spacing': 1.1312499999999992}
0.0(3.1) 0.0(5.7)
{'chiral': 0.5749999999999988,'spacing': 1.0687499999999992}
0.0(2.9) 0.0(5.3)
{'chiral': 0.4124999999999983,'spacing': 1.065624999999999}
0.0(2.1) 0.0(5.3)
{'chiral': 0.18749999999999778,'spacing': 1.1843749999999984}
0.00(94) 0.0(5.9)
{'chiral': -0.14375000000000315,'spacing': 1.270312499999

{'chiral': -0.14375000000000315,'spacing': 1.2703124999999975}
-0.00(72) 0.0(6.4)
{'chiral': -0.02500000000000302,'spacing': 1.1187499999999981}
-0.00(13) 0.0(5.6)
{'chiral': 0.4624999999999986,'spacing': 1.128124999999999}
0.0(2.3) 0.0(5.6)
{'chiral': 0.1374999999999975,'spacing': 1.1218749999999984}
0.00(69) 0.0(5.6)
{'chiral': 0.3812499999999983,'spacing': 1.1265624999999988}
0.0(1.9) 0.0(5.6)
{'chiral': 0.15624999999999778,'spacing': 1.245312499999998}
0.00(78) 0.0(6.2)
{'chiral': 0.3484374999999982,'spacing': 1.1105468749999987}
0.0(1.7) 0.0(5.6)
{'chiral': 0.15468749999999765,'spacing': 1.1683593749999985}
0.00(77) 0.0(5.8)
{'chiral': 0.32460937499999815,'spacing': 1.1370117187499988}
0.0(1.6) 0.0(5.7)
{'chiral': 0.4855468749999985,'spacing': 1.063183593749999}
0.0(2.4) 0.0(5.3)
{'chiral': 0.26201171874999796,'spacing': 1.1540771484374985}
0.0(1.3) 0.0(5.8)
{'chiral': 0.23818359374999787,'spacing': 1.1805419921874984}
0.0(1.2) 0.0(5.9)
{'chiral': 0.17558593749999768,'spacing': 1.

{'chiral': 0.23867870212468,'spacing': 0.18455232449384296}
0.0(1.2) 0.00(92)
{'chiral': 0.2386210549605865,'spacing': 0.18462009703742432}
0.0(1.2) 0.00(92)
{'chiral': 0.2386210549605865,'spacing': 0.18462009703742432}
0.0(1.2) 0.00(92)
Making fits...
Done!
100% complete Time (s):  0.706000089645
Compiling results...
Saving...
Done.

Total time (s):  37.0780000687 

FKFpi_alphaS
(model, F2, Vol, alphaS, logSq): ('ma', 'FKFpi', False, True, False)
Using default prior.
{'chiral': 1.0,'spacing': 1.0}
0.0(5.0) 0.0(5.0)
{'chiral': 1.05,'spacing': 1.0}
0.0(5.2) 0.0(5.0)
{'chiral': 1.0,'spacing': 1.05}
0.0(5.0) 0.0(5.2)
{'chiral': 0.95,'spacing': 1.0499999999999998}
0.0(4.8) 0.0(5.2)
{'chiral': 0.8999999999999999,'spacing': 1.0749999999999997}
0.0(4.5) 0.0(5.4)
{'chiral': 0.8999999999999999,'spacing': 1.0249999999999997}
0.0(4.5) 0.0(5.1)
{'chiral': 0.8499999999999996,'spacing': 1.0124999999999997}
0.0(4.2) 0.0(5.1)
{'chiral': 0.7499999999999996,'spacing': 1.0874999999999995}
0.0(3.7) 0.0(5.

{'chiral': 0.1934921872818864,'spacing': -0.16934684808421202}
0.00(97) -0.00(85)
{'chiral': 0.19345700721421527,'spacing': -0.16929927264741731}
0.00(97) -0.00(85)
{'chiral': 0.19345539304314063,'spacing': -0.16936486066500298}
0.00(97) -0.00(85)
{'chiral': 0.19345539304314063,'spacing': -0.16936486066500298}
0.00(97) -0.00(85)
Making fits...
Done!
100% complete Time (s):  1.02699995041
Compiling results...
Saving...
Done.

Total time (s):  44.4029998779 

FKFpi_alphaS_logSq
(model, F2, Vol, alphaS, logSq): ('ma', 'FKFpi', False, True, True)
Using default prior.
{'chiral': 1.0,'spacing': 1.0}
0.0(5.0) 0.0(5.0)
{'chiral': 1.05,'spacing': 1.0}
0.0(5.2) 0.0(5.0)
{'chiral': 1.0,'spacing': 1.05}
0.0(5.0) 0.0(5.2)
{'chiral': 0.95,'spacing': 1.0499999999999998}
0.0(4.8) 0.0(5.2)
{'chiral': 0.8999999999999999,'spacing': 1.0749999999999997}
0.0(4.5) 0.0(5.4)
{'chiral': 0.8999999999999999,'spacing': 1.0249999999999997}
0.0(4.5) 0.0(5.1)
{'chiral': 0.8499999999999996,'spacing': 1.012499999999999

{'chiral': 0.23857493790231127,'spacing': 0.15408624433790172}
0.0(1.2) 0.00(77)
{'chiral': 0.23818700171899687,'spacing': 0.15430984482701227}
0.0(1.2) 0.00(77)
{'chiral': 0.238580968941304,'spacing': 0.15378776879935274}
0.0(1.2) 0.00(77)
{'chiral': 0.23883155167818998,'spacing': 0.15403666919134434}
0.0(1.2) 0.00(77)
{'chiral': 0.23870475254999882,'spacing': 0.1539868378799858}
0.0(1.2) 0.00(77)
{'chiral': 0.2386987215110061,'spacing': 0.1542853134185348}
0.0(1.2) 0.00(77)
{'chiral': 0.23861040708372955,'spacing': 0.15391215495414826}
0.0(1.2) 0.00(77)
{'chiral': 0.23863984522615506,'spacing': 0.15403654110894377}
0.0(1.2) 0.00(77)
{'chiral': 0.23864286074565141,'spacing': 0.15388730333966927}
0.0(1.2) 0.00(77)
{'chiral': 0.2387017370305025,'spacing': 0.1541360756492603}
0.0(1.2) 0.00(77)
{'chiral': 0.2386870179592897,'spacing': 0.15407388257186255}
0.0(1.2) 0.00(77)
{'chiral': 0.23867229888807695,'spacing': 0.15401168949446478}
0.0(1.2) 0.00(77)
{'chiral': 0.23864135298590322,'spac

{'chiral': 0.18910642545669937,'spacing': 0.1696095263119775}
0.00(95) 0.00(85)
{'chiral': 0.189983319805466,'spacing': 0.1680988067033456}
0.00(95) 0.00(84)
{'chiral': 0.18969102168921045,'spacing': 0.1686023799062229}
0.00(95) 0.00(84)
{'chiral': 0.18985005053691228,'spacing': 0.1680673897615622}
0.00(95) 0.00(84)
{'chiral': 0.18958351199980486,'spacing': 0.16800455587799545}
0.00(95) 0.00(84)
{'chiral': 0.18961038942215624,'spacing': 0.16815401188505233}
0.00(95) 0.00(84)
{'chiral': 0.189184822037347,'spacing': 0.16862616814614628}
0.00(95) 0.00(84)
{'chiral': 0.18968374341202096,'spacing': 0.16820708435770823}
0.00(95) 0.00(84)
{'chiral': 0.18951743628712964,'spacing': 0.16834677895385425}
0.00(95) 0.00(84)
{'chiral': 0.18963726684450766,'spacing': 0.16830346789210918}
0.00(95) 0.00(84)
{'chiral': 0.189304652594725,'spacing': 0.1685828570844012}
0.00(95) 0.00(84)
{'chiral': 0.189554113282062,'spacing': 0.16837331519018217}
0.00(95) 0.00(84)
{'chiral': 0.18947095971961633,'spacing':

{'chiral': 0.1894244831251352,'spacing': 0.1685395460418906}
0.00(95) 0.00(84)
{'chiral': 0.18942448312753823,'spacing': 0.16853954605525298}
0.00(95) 0.00(84)
{'chiral': 0.18942448312673724,'spacing': 0.16853954605079885}
0.00(95) 0.00(84)
{'chiral': 0.18942448314022114,'spacing': 0.16853954604118163}
0.00(95) 0.00(84)
{'chiral': 0.1894244831386191,'spacing': 0.16853954603227336}
0.00(95) 0.00(84)
{'chiral': 0.18942448313982063,'spacing': 0.16853954603895455}
0.00(95) 0.00(84)
{'chiral': 0.18942448313942012,'spacing': 0.1685395460367275}
0.00(95) 0.00(84)
{'chiral': 0.1894244831461621,'spacing': 0.1685395460319189}
0.00(95) 0.00(84)
{'chiral': 0.18942448314536106,'spacing': 0.16853954602746474}
0.00(95) 0.00(84)
{'chiral': 0.18942448314596183,'spacing': 0.16853954603080534}
0.00(95) 0.00(84)
{'chiral': 0.18942448314556132,'spacing': 0.1685395460285783}
0.00(95) 0.00(84)
{'chiral': 0.1894244831458617,'spacing': 0.16853954603024857}
0.00(95) 0.00(84)
{'chiral': 0.18942448314566146,'spac

KeyboardInterrupt: 